# Implementing a CNN from scratch

In [ ]:
from fastai2.vision import 

In [1]:
from collections.abc import Iterable

In [2]:
import torch
from torch import nn
import torch.nn.functional as F

## eg 

In [3]:
im = torch.randn((5, 3, 64, 64))

In [4]:
weights = torch.randn((32, 3, 3, 3))

In [5]:
unf = nn.Unfold((3,3), padding=0, stride=2)

In [6]:
im_unf = unf(im)
im_unf.shape

torch.Size([5, 27, 961])

In [7]:
im_unf.transpose_(1,2)
im_unf.shape

torch.Size([5, 961, 27])

In [8]:
weights.view(weights.shape[0], -1).T.shape

torch.Size([27, 32])

In [9]:
convolve = im_unf @ weights.view(weights.shape[0], -1).T
convolve.shape

torch.Size([5, 961, 32])

In [12]:
fold = nn.Fold((31, 31), (1, 1))

In [13]:
results = fold(convolve.transpose(1,2))
results.shape

torch.Size([5, 32, 31, 31])

## Real

**check how bias works and add bias**

In [18]:
class MyConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super().__init__()
        self.in_channels, self.out_channels = in_channels, out_channels
        self.kernel_size = self._pair(kernel_size)
        self.stride, self.padding = stride, padding
        self.weight = nn.Parameter(torch.randn((out_channels, in_channels, *self.kernel_size)))
        
    def forward(self, x):
        assert(x.shape[1] == self.in_channels)
        unf = F.unfold(x, self.kernel_size, padding=self.padding, stride=self.stride)
        # reshape
        unf.transpose_(1, 2)
        # convolve image with the filters
        conv = (unf @ self.weight.view(self.weight.shape[0], -1).T).transpose(1, 2)
        # fold back to appropriate size
        oz = self.output_size(x.shape[2], x.shape[3])
        return F.fold(conv, oz, (1,1))
        
    def output_size(self, x_r, x_c):
        oz_r = self.cal_output_dim(x_r, self.kernel_size[0])
        # check if the image and filters are square
        if x_r == x_c and self.kernel_size[0] == self.kernel_size[1]:
            return oz_r, oz_r
        oz_c = self.cal_output_dim(x_c, self.kernel_size[1])
        return oz_r, oz_c
    
    def cal_output_dim(self, n, kernel_size):
        return (n + 2 * self.padding - kernel_size) // self.stride + 1
        
    def _pair(self, k):
        # utility function to allow the use of a single integer for the kernel size
        if isinstance(k, Iterable):
            return k
        return tuple((k, k))

In [19]:
my_conv1 = MyConv2d(3, 16, 3)

In [20]:
im = torch.randn((1, 3, 28, 28))

In [21]:
res = my_conv1(im)

In [23]:
res.shape

torch.Size([1, 16, 26, 26])

## Train model with it

## stop using nn.Parameter and implement backprop from scratch